In [1]:
import numpy as np
import pandas as pd 
from datetime import datetime
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
df = pd.read_csv('wiki_data.csv')
df=df[df.columns[1:]]
nan_values = df[df.columns[:26]].isna().sum().sort_values()
columns_keeping = nan_values[nan_values<6000].index
df = df[columns_keeping]
df['Based on'] = df['Based on'].apply(lambda x: 0 if pd.isna(x) else 1)

C:\Users\jacob\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (36,48,68,69,71,72,75,78,82,100,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,134,135,136,137,138,139,140,141,145,146,147,148,149,150,151,155,158,159,165,166,167,168,169,170,171,172,173,175,176,177,178,179,180,181,182,183,184,185,186,187,188,194,195,196,200,201,202,204,205,206,207,208,209,210,211,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
for col in df.columns[1:]:
    try:
        df[col] = df[col].str[3:-3]
    except:
        print(col)
        

Based on


In [4]:
df['Running time'] = df['Running time'].str.extract(r'(\d+)')
df['Running time'] = pd.to_numeric(df['Running time'])

## making all these columns into list becasue they have mulitple names and will make creating dummy variables for each
## later in the cleaning process
df['Directed by'] = df['Directed by'].str.split(',')
df['Starring'] = df['Starring'].str.split(',')
df['Produced by'] = df['Produced by'].str.split(',')
df['Screenplay by'] = df['Screenplay by'].str.split(',')
df['Edited by'] = df['Edited by'].str.split(',')
df['Cinematography'] = df['Cinematography'].str.split(',')
df['Written by'] = df['Written by'].str.split(',')
df['Music by'] = df['Music by'].str.split(',')
df['Productioncompanies '] = df['Productioncompanies '].str.split(',')
df['Distributed by'] = df['Distributed by'].str.split(',')
df['Productioncompany '] = df['Productioncompany '].str.split(',')

In [5]:
df['Release date'] = df['Release date'].str.split('(') 
df['Release date length'] = df['Release date'].apply(lambda x:len(x) if type(x) == list else x)
df['Release date'] =  df['Release date'][df['Release date length']!=10]

In [6]:
 def clean_release(row):
        if type(row)==list:
            if len(row)==1:
                return row[0]
            else:
                return row[1][:-1]
        else:
            row
            
df['Release date']=df['Release date'].apply(clean_release).str.split(')').str[0]

In [7]:
def fix_dates (row):
    if pd.isna(row):
        return row
    else:
        try : 
            return datetime.strptime(row, "%Y-%m-%d")
        except:
            try:
                return datetime.strptime(row, "%B %d, %Y")
            except:
                return row
    

df['Release date'] = df['Release date'].apply(fix_dates)

In [8]:
import re 
df = df[df['Box office'].str.contains('$', na=False, regex=False)]  # remove any non dollar currency
df =df[~df['Box office'].str.contains('admission',na=False,regex=False)]#removes the 5 rows with admission in the box office col

def clean_box(row):
    box = row[1:]
    box =box.replace('$','')
    box = re.split(" |\(|\[", box)
    box = box[0].split('\\')[0]
    
    if bool(re.match(r".*million.*", row)):
        try: return float(box)*1000000
        except: return np.nan
    elif bool(re.match(r".*billion.*", row)):
        try: return float(box)*1000000000
        except: return np.nan  
        
    else: 
        try: return float(box.replace(',',''))
        except: np.nan
    
    

df['Box office'] = df['Box office'].apply(clean_box)

In [9]:
df['million'] = df['Budget'].str.contains('millio',na=False,regex=False)
df['Budget'] = df['Budget'].str[1:].apply(lambda x: re.split(' |\[', str(x))).str[0].str.replace(',','').str.split('\\').str[0]


In [10]:
def clean_budget(x):
    if '–' in x:
        try: return (float(x.replace(',','').split('–')[0])+float(x.replace(',','').split('–')[1]))/2 
        except: return np.nan
    else:
        try: return float(x)
        except: np.nan
df['Budget'] = df['Budget'].apply(clean_budget)

In [11]:
def million(x):
    budget = x['Budget']
    if x['million'] == True:
        return budget*1000000
    else:
        return budget
df['Budget'] = df.apply(million,axis=1)

In [12]:
df = df.drop(columns=['Release date length','million'])

In [13]:
def clean_names(row):
    row_list = []
    if type(row) == list:
        for word in row:
            word = word.replace("'",'')
            word = word.replace('"','')
            word = word.strip()
            if len(word) <40:
                row_list.append(word)
        return row_list
    else: return np.nan
    
df['Starring'] = df['Starring'].apply(clean_names)
df['Produced by'] = df['Produced by'].apply(clean_names)
df['Edited by'] = df['Edited by'].apply(clean_names)
df['Cinematography'] = df['Cinematography'].apply(clean_names)
df['Directed by'] = df['Directed by'].apply(clean_names)
df['Written by'] = df['Written by'].apply(clean_names)
df['Productioncompanies '] = df['Productioncompanies '].apply(clean_names)
df['Music by'] = df['Music by'].apply(clean_names)
df['Screenplay by'] = df['Screenplay by'].apply(clean_names)
df['Distributed by'] = df['Distributed by'].apply(clean_names)
df['Productioncompany '] = df['Productioncompany '].apply(clean_names)

In [14]:
# df['Productioncompany '][df['Productioncompany '].str.split(',').apply(lambda x: len(x) if type(x) == list else x)==3]

In [15]:
df.head()

,Title,Running time,Directed by,Starring,Release date,Produced by,Distributed by,Music by,Edited by,Cinematography,Language,Box office,Country,Budget,Written by,Productioncompanies,Screenplay by,Based on,Productioncompany
0,The Adventures of Ford Fairlane,102.0,[Renny Harlin],"[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",1990-07-11 00:00:00,"[Steve Perry, Joel Silver]",[20th Century Fox],[Yello],[Michael Tronick],[Oliver Wood],English,21400000.0,United States,20000000.0,NaN,NaN,"[David Arnott, James Cappe, Daniel Waters]",1,[Silver Pictures]
1,"After Dark, My Sweet",114.0,[James Foley],"[Jason Patric, Rachel Ward, Bruce Dern, George...",1990-05-17 00:00:00,[Ric KidneyRobert Redlin],[Avenue Pictures],[Maurice Jarre],[Howard E. Smith],[Mark Plummer],English,2700000.0,United States,6000000.0,NaN,NaN,[James FoleyRobert Redlin],1,[Avenue Pictures]
2,Air America,113.0,[Roger Spottiswoode],"[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",1990-08-10 00:00:00,[Daniel MelnickMario Kassar],[TriStar Pictures],[Charles Gross],[John BloomLois Freeman-Fox],[Roger Deakins],English,57700000.0,United States,32500000.0,NaN,[Carolco PicturesIndieProd Company],"[John Eskow, Richard Rush]",1,NaN
3,Alice,106.0,[Woody Allen],"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",1990-12-25 00:00:00,[Robert Greenhut],[Orion Pictures],NaN,[Susan E. Morse],[Carlo Di Palma],English,7331647.0,United States,12000000.0,[Woody Allen],NaN,NaN,0,NaN
4,Almost an Angel,95.0,[John Cornell],"[Paul Hogan, Elias Koteas, Linda Kozlowski]",1990-12-19 00:00:00,[John Cornell],[Paramount Pictures],[Maurice Jarre],[David Stiven],[Russell Boyd],English,6939946.0,United States,25000000.0,[Paul Hogan],NaN,NaN,0,NaN


In [16]:
cast = {}
for i in df['Produced by']:
    if type(i) == list:
        for j in i:
            if j not in cast:
                cast[j] = 1
            else:
                cast[j] += 1

In [17]:
pd.DataFrame.from_dict(cast,orient='index').sort_values(by=0, ascending=False).head(4)

,0
Scott Rudin,56
Brian Grazer,52
Neal H. Moritz,42
Joel Silver,41


In [18]:
# dummy_df = df.copy()
# dummy_df = dummy_df.reset_index().drop(columns='index')

# mlb = MultiLabelBinarizer()
# starring_nan = dummy_df[dummy_df['Starring'].isna()]
# dummy_df = dummy_df[~dummy_df['Starring'].isna()]

# dummy_df = dummy_df.join(pd.DataFrame(mlb.fit_transform(dummy_df.pop('Starring')),
#                           columns=mlb.classes_,
#                           index=dummy_df.index))

In [19]:
# dummy_df.head()